In [1]:
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime

In [2]:
# データ読み込み
import urbansound8k_loader as dataset

#parent_dir = 'UrbanSound8K/audio/'
#tr_features, tr_labels = dataset.load_urbansound8k(parent_dir, ['fold1', 'fold2'])
#ts_features, ts_labels = dataset.load_urbansound8k(parent_dir, ['fold3'])

%time tr_features, tr_labels, ts_features, ts_labels = dataset.load_from_npy_files('all_')

if (len(tr_features) !=  len(tr_labels)):
    print('WARN: invalid # of training data. features=' + str(len(tr_features)) + ', labels=' + str(len(tr_labels)))
elif (len(ts_features) !=  len(ts_labels)):
    print('WARN: invalid # of tast data. features=' + str(len(ts_features)) + ', labels=' + str(len(ts_labels)))
elif (len(tr_features) == 0 or len(ts_features) == 0):
    print('WARN: no data.')
else: 
    print('loaded successfully. # of train data=' + str(len(tr_features)) + ', # of test data=' + str(len(ts_features)))


CPU times: user 2.4 ms, sys: 19.7 ms, total: 22.1 ms
Wall time: 25.1 ms
loaded successfully. # of train data=6269, # of test data=6269


In [94]:
training_epochs = 5000
n_dim = tr_features.shape[1]  # 193
n_classes = 10
n_hidden_units_one = 280 
n_hidden_units_two = 300

In [95]:
# モデル構築、学習、評価 by keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import keras.callbacks
import keras.backend.tensorflow_backend as KTF

log_filepath = './keras/log'
batch_size = 128
old_session = KTF.get_session()

with tf.Graph().as_default():
    session = tf.Session('')
    KTF.set_session(session)
    KTF.set_learning_phase(1)
    
    model = Sequential()
    model.add(Dense(n_hidden_units_one, activation='relu', input_shape=(n_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(n_hidden_units_two, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
    cbks = [tb_cb]
    
    print ('fit start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    history = model.fit(tr_features, tr_labels,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1,
                        callbacks=cbks, 
                        validation_data=(ts_features, ts_labels))

    print ('evaluate start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    score = model.evaluate(ts_features, ts_labels, verbose=0)
    print('Test accuracy:', score[1])
    print ('finished: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

KTF.set_session(old_session)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 280)               54320     
_________________________________________________________________
dropout_1 (Dropout)          (None, 280)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               84300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3010      
Total params: 141,630
Trainable params: 141,630
Non-trainable params: 0
_________________________________________________________________
fit start:  2017/07/02 16:15:53
Train on 1757 samples, validate on 925 samples
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal;

1757/1757 [==============================] - 0s - loss: 0.1144 - acc: 0.9880 - val_loss: 7.3927 - val_acc: 0.3665
Epoch 116/5000
1757/1757 [==============================] - 0s - loss: 0.1071 - acc: 0.9863 - val_loss: 7.4494 - val_acc: 0.3697
Epoch 117/5000
1757/1757 [==============================] - 0s - loss: 0.1017 - acc: 0.9915 - val_loss: 7.7203 - val_acc: 0.3557
Epoch 118/5000
1757/1757 [==============================] - 0s - loss: 0.1096 - acc: 0.9886 - val_loss: 7.6705 - val_acc: 0.3546
Epoch 119/5000
1757/1757 [==============================] - 0s - loss: 0.1064 - acc: 0.9920 - val_loss: 7.4939 - val_acc: 0.3438
Epoch 120/5000
1757/1757 [==============================] - 0s - loss: 0.1020 - acc: 0.9915 - val_loss: 7.4286 - val_acc: 0.3903
Epoch 121/5000
1757/1757 [==============================] - 0s - loss: 0.1203 - acc: 0.9875 - val_loss: 7.7529 - val_acc: 0.3470
Epoch 122/5000
1757/1757 [==============================] - 0s - loss: 0.1026 - acc: 0.9898 - val_loss: 7.9220 -

1757/1757 [==============================] - 0s - loss: 0.0973 - acc: 0.9915 - val_loss: 8.1337 - val_acc: 0.3935
Epoch 242/5000
1757/1757 [==============================] - 0s - loss: 0.1065 - acc: 0.9898 - val_loss: 7.8516 - val_acc: 0.4086
Epoch 243/5000
1757/1757 [==============================] - 0s - loss: 0.0943 - acc: 0.9926 - val_loss: 7.9162 - val_acc: 0.3903
Epoch 244/5000
1757/1757 [==============================] - 0s - loss: 0.1171 - acc: 0.9903 - val_loss: 7.9081 - val_acc: 0.4011
Epoch 245/5000
1757/1757 [==============================] - 0s - loss: 0.1113 - acc: 0.9909 - val_loss: 7.9376 - val_acc: 0.3935
Epoch 246/5000
1757/1757 [==============================] - 0s - loss: 0.0938 - acc: 0.9937 - val_loss: 7.8884 - val_acc: 0.4011
Epoch 247/5000
1757/1757 [==============================] - 0s - loss: 0.0930 - acc: 0.9943 - val_loss: 7.9807 - val_acc: 0.3989
Epoch 248/5000
1757/1757 [==============================] - 0s - loss: 0.1040 - acc: 0.9909 - val_loss: 8.0623 -

1757/1757 [==============================] - 0s - loss: 0.1013 - acc: 0.9915 - val_loss: 8.5051 - val_acc: 0.3805
Epoch 368/5000
1757/1757 [==============================] - 0s - loss: 0.1190 - acc: 0.9903 - val_loss: 8.3505 - val_acc: 0.3978
Epoch 369/5000
1757/1757 [==============================] - 0s - loss: 0.0921 - acc: 0.9943 - val_loss: 8.5568 - val_acc: 0.3751
Epoch 370/5000
1757/1757 [==============================] - 0s - loss: 0.1033 - acc: 0.9909 - val_loss: 8.2753 - val_acc: 0.4032
Epoch 371/5000
1757/1757 [==============================] - 0s - loss: 0.0999 - acc: 0.9909 - val_loss: 8.3643 - val_acc: 0.3784
Epoch 372/5000
1757/1757 [==============================] - 0s - loss: 0.1042 - acc: 0.9909 - val_loss: 8.1762 - val_acc: 0.3978
Epoch 373/5000
1757/1757 [==============================] - 0s - loss: 0.1239 - acc: 0.9892 - val_loss: 8.2696 - val_acc: 0.3870
Epoch 374/5000
1757/1757 [==============================] - 0s - loss: 0.1002 - acc: 0.9926 - val_loss: 8.2475 -

1757/1757 [==============================] - 0s - loss: 0.0895 - acc: 0.9943 - val_loss: 8.8078 - val_acc: 0.3946
Epoch 494/5000
1757/1757 [==============================] - 0s - loss: 0.1128 - acc: 0.9915 - val_loss: 8.6420 - val_acc: 0.3838
Epoch 495/5000
1757/1757 [==============================] - 0s - loss: 0.0968 - acc: 0.9915 - val_loss: 8.4225 - val_acc: 0.4119
Epoch 496/5000
1757/1757 [==============================] - 0s - loss: 0.1217 - acc: 0.9920 - val_loss: 8.7644 - val_acc: 0.3859
Epoch 497/5000
1757/1757 [==============================] - 0s - loss: 0.1042 - acc: 0.9903 - val_loss: 8.6893 - val_acc: 0.3676
Epoch 498/5000
1757/1757 [==============================] - 0s - loss: 0.0907 - acc: 0.9915 - val_loss: 8.3566 - val_acc: 0.3903
Epoch 499/5000
1757/1757 [==============================] - 0s - loss: 0.0930 - acc: 0.9932 - val_loss: 8.3896 - val_acc: 0.4022
Epoch 500/5000
1757/1757 [==============================] - 0s - loss: 0.0829 - acc: 0.9949 - val_loss: 8.2666 -

...

1757/1757 [==============================] - 0s - loss: 0.1083 - acc: 0.9926 - val_loss: 9.1437 - val_acc: 0.4054
Epoch 4400/5000
1757/1757 [==============================] - 0s - loss: 0.0900 - acc: 0.9937 - val_loss: 9.3141 - val_acc: 0.3946
Epoch 4401/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9.3259 - val_acc: 0.4000
Epoch 4402/5000
1757/1757 [==============================] - 0s - loss: 0.0877 - acc: 0.9943 - val_loss: 9.0666 - val_acc: 0.4151
Epoch 4403/5000
1757/1757 [==============================] - 0s - loss: 0.0879 - acc: 0.9943 - val_loss: 9.0851 - val_acc: 0.4022
Epoch 4404/5000
1757/1757 [==============================] - 0s - loss: 0.0938 - acc: 0.9937 - val_loss: 9.5476 - val_acc: 0.3827
Epoch 4405/5000
1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 9.5101 - val_acc: 0.3827
Epoch 4406/5000
1757/1757 [==============================] - 0s - loss: 0.0853 - acc: 0.9932 - val_loss: 9

1757/1757 [==============================] - 0s - loss: 0.0891 - acc: 0.9937 - val_loss: 8.9318 - val_acc: 0.4216
Epoch 4526/5000
1757/1757 [==============================] - 0s - loss: 0.1020 - acc: 0.9932 - val_loss: 8.8198 - val_acc: 0.4227
Epoch 4527/5000
1757/1757 [==============================] - 0s - loss: 0.1021 - acc: 0.9932 - val_loss: 9.0991 - val_acc: 0.4054
Epoch 4528/5000
1757/1757 [==============================] - 0s - loss: 0.0918 - acc: 0.9943 - val_loss: 8.9640 - val_acc: 0.4205
Epoch 4529/5000
1757/1757 [==============================] - 0s - loss: 0.0991 - acc: 0.9926 - val_loss: 9.0169 - val_acc: 0.4173
Epoch 4530/5000
1757/1757 [==============================] - 0s - loss: 0.0883 - acc: 0.9943 - val_loss: 9.1018 - val_acc: 0.4022
Epoch 4531/5000
1757/1757 [==============================] - 0s - loss: 0.0918 - acc: 0.9943 - val_loss: 8.9650 - val_acc: 0.4141
Epoch 4532/5000
1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 8

1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 9.2128 - val_acc: 0.4076
Epoch 4652/5000
1757/1757 [==============================] - 0s - loss: 0.1141 - acc: 0.9920 - val_loss: 9.5852 - val_acc: 0.3795
Epoch 4653/5000
1757/1757 [==============================] - 0s - loss: 0.0873 - acc: 0.9943 - val_loss: 9.5070 - val_acc: 0.3870
Epoch 4654/5000
1757/1757 [==============================] - 0s - loss: 0.1007 - acc: 0.9937 - val_loss: 9.4813 - val_acc: 0.3827
Epoch 4655/5000
1757/1757 [==============================] - 0s - loss: 0.1009 - acc: 0.9937 - val_loss: 9.5697 - val_acc: 0.3838
Epoch 4656/5000
1757/1757 [==============================] - 0s - loss: 0.1041 - acc: 0.9920 - val_loss: 9.4460 - val_acc: 0.3924
Epoch 4657/5000
1757/1757 [==============================] - 0s - loss: 0.0930 - acc: 0.9937 - val_loss: 9.6097 - val_acc: 0.3784
Epoch 4658/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9

1757/1757 [==============================] - 0s - loss: 0.0889 - acc: 0.9937 - val_loss: 8.9273 - val_acc: 0.4162
Epoch 4778/5000
1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 8.6042 - val_acc: 0.4368
Epoch 4779/5000
1757/1757 [==============================] - 0s - loss: 0.1001 - acc: 0.9937 - val_loss: 8.9835 - val_acc: 0.4130
Epoch 4780/5000
1757/1757 [==============================] - 0s - loss: 0.0858 - acc: 0.9943 - val_loss: 9.0626 - val_acc: 0.4065
Epoch 4781/5000
1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 8.9597 - val_acc: 0.4249
Epoch 4782/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 8.7508 - val_acc: 0.4270
Epoch 4783/5000
1757/1757 [==============================] - 0s - loss: 0.0849 - acc: 0.9943 - val_loss: 9.1416 - val_acc: 0.3978
Epoch 4784/5000
1757/1757 [==============================] - 0s - loss: 0.1009 - acc: 0.9937 - val_loss: 9

1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9.0317 - val_acc: 0.4130
Epoch 4904/5000
1757/1757 [==============================] - 0s - loss: 0.1044 - acc: 0.9926 - val_loss: 9.4867 - val_acc: 0.3881
Epoch 4905/5000
1757/1757 [==============================] - 0s - loss: 0.0837 - acc: 0.9943 - val_loss: 8.9899 - val_acc: 0.4173
Epoch 4906/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9.1928 - val_acc: 0.4076
Epoch 4907/5000
1757/1757 [==============================] - 0s - loss: 0.0958 - acc: 0.9932 - val_loss: 9.2156 - val_acc: 0.4022
Epoch 4908/5000
1757/1757 [==============================] - 0s - loss: 0.0917 - acc: 0.9943 - val_loss: 9.3657 - val_acc: 0.3968
Epoch 4909/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9.1104 - val_acc: 0.4086
Epoch 4910/5000
1757/1757 [==============================] - 0s - loss: 0.0826 - acc: 0.9949 - val_loss: 9